<a href="https://colab.research.google.com/github/boeyjw/kaggle-store/blob/main/CodeStore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logodds

In [ ]:
def logodds(x):
    if not len(x):
        return 0
    event_dist = sum(x["smoking"]) / len(x)
    non_event_dist = sum(x["smoking"] == 0) / len(x)

    return np.log((event_dist + 0.001) / (non_event_dist + 0.001))

## WoE Transformer

In [ ]:
class WOETransformer(BaseEstimator, TransformerMixin, ClassNamePrefixFeaturesOutMixin):
    def __init__(self, eps: float = 0.001):
        self.woe_mapping_ = {}
        self.eps = eps

    def __sklearn_clone__(self):
        return self


    def fit(self, X, y=None):
        def logodds(x):
            if not len(x):
                return 0
            event_dist = sum(x) / len(x)
            non_event_dist = sum(x == 0) / len(x)

            return np.log((event_dist + self.eps) / (non_event_dist + self.eps))
        # check_X_y(X, y, accept_sparse=False)

        Xy = pd.concat([X, y.astype(int).to_frame("target")], axis=1)
        self.feature_names_in_ = X.columns.tolist()
        self.n_features_ = X.shape[1]

        del X, y

        for c in self.feature_names_in_:
            self.woe_mapping_[c] = Xy.groupby(c)["target"].apply(logodds).to_dict()

        self._n_features_out = len(self.woe_mapping_)
        self.is_fitted_ = True

        return self

    def transform(self, X):
        check_is_fitted(self, "is_fitted_")
        if X.shape[1] != self.n_features_:
            raise ValueError('Shape of input is different from what was seen'
                             'in `fit`')

        for c in X.columns:
            X.loc[:, c] = X[c].map(self.woe_mapping_[c])

        return X.values

    def get_feature_names_out(self, input_features=None):
        return [c for c in self.feature_names_in_]

class MulticlassWOETransformer(BaseEstimator, TransformerMixin, ClassNamePrefixFeaturesOutMixin):
    def __init__(self, eps: float = 0.001):
        self.woe_mapping_ = {}
        self.eps = eps

    def __sklearn_clone__(self):
        return self

    def fit(self, X, y=None):
        def logodds(x):
            if not len(x):
                return 0
            event_dist = sum(x) / len(x)
            non_event_dist = sum(x == 0) / len(x)

            return np.log((event_dist + self.eps) / (non_event_dist + self.eps))
        # check_X_y(X, y, accept_sparse=False)

        yy = y.copy(deep=True)
        del y

        yy = yy.to_frame("target")
        for i in yy["target"].unique():
            yy[f"target_{i}"] = np.where(yy["target"] == i, 1, 0)
        Xy = pd.concat([X, yy.drop(columns=["target"])], axis=1)
        self.feature_names_in_ = X.columns.tolist()
        self.n_features_ = X.shape[1]

        for c in self.feature_names_in_:
            for t in yy.drop(columns=["target"]).columns:
                self.woe_mapping_[c + "_" + t.split("_")[1]] = Xy.groupby(c)[t].apply(logodds).to_dict()

        self._n_features_out = len(self.woe_mapping_)
        self.is_fitted_ = True

        return self

    def transform(self, X):
        check_is_fitted(self, "is_fitted_")
        if X.shape[1] != self.n_features_:
            raise ValueError('Shape of input is different from what was seen'
                             'in `fit`')

        X = X.assign(**{c: X["_".join(c.split("_")[:-1])].map(v) for c, v in self.woe_mapping_.items()})

        return X[list(self.woe_mapping_.keys())].values

    def get_feature_names_out(self, input_features=None):
        return [c for c in self.woe_mapping_]

# Sklearn Custom Classes

## Pipeline

In [ ]:
class FitExposedPipeline(Pipeline):
    """
    Exposes final pipeline estimator coef_ or feature_importances_
    """
    @property
    def coef_(self):
        return self._final_estimator.coef_
    @property
    def feature_importances_(self):
        return self._final_estimator.feature_importances_